In [1]:
# imports
from sqlalchemy import create_engine, inspect
import geopandas as gpd
from meteostat import Stations, Daily, Point
from helper import get_nuts_data, get_daily_weather_data, plot_centroids_with_radius
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import logging
import warnings
from logging.handlers import RotatingFileHandler

%matplotlib qt

In [2]:
# redirect logger
logger_file_handler = RotatingFileHandler(u'/dev/null')
logger_file_handler.setLevel(logging.WARNING)
logging.captureWarnings(True)
root_logger = logging.getLogger()
root_logger.addHandler(logger_file_handler)
root_logger.setLevel(logging.WARNING)

In [3]:
path = "weather.db"

In [4]:
engine = create_engine("sqlite:///"+path, echo=False)

In [5]:
# Check which tables already exist
insp = inspect(engine)
tables = insp.get_table_names()

In [6]:
# add nuts data table
for lvl in range(1, 4):
    if f'nuts{lvl}' in tables:
        continue
    print(f"Reading NUTS{lvl} data")
    data = get_nuts_data(lvl)
    data.set_index(["NUTS_CODE"], inplace=True)
    data.to_sql(f'nuts{lvl}', con=engine, if_exists='replace')

In [7]:
# add centroid weather data
lvl = 3
df = pd.read_sql_table(f'nuts{lvl}', engine)
df.head()

,NUTS_CODE,NUTS_LEVEL,NUTS_NAME,lon_times100,lat_times100
0,DE111,3,"Stuttgart, Stadtkreis",917,4877
1,DE112,3,Böblingen,894,4867
2,DE113,3,Esslingen,936,4864
3,DE114,3,Göppingen,971,4866
4,DE115,3,Ludwigsburg,912,4894


In [8]:
radius = 50000
start = datetime(year=1984, month=1, day=1)
end = datetime(year=2021, month=1, day=1)
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    index = row['NUTS_CODE'] + '_weather'
    if index in tables:
        continue
    lat, lon = row['lat_times100'] / 100, row['lon_times100'] / 100
    daily = get_daily_weather_data(lat, lon, radius, start, end)
    # write data to database
    if daily.shape[0] > 0:
        daily.to_sql(index, con=engine, if_exists='replace')

100%|██████████████████████████████████████| 401/401 [00:00<00:00, 20904.27it/s]


In [9]:
plot_centroids_with_radius(3)